In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (12,12)
mineProbability = 0.2      # Probability that a square contain a mine
missingProbability = 0.1   # Probability that a square is missing adjacency info

In [3]:
# count the number of mines in the proximity of given square, including square itself
def countMines(board,r,c):
    count = 0
    rows, cols = board.shape
    for i in [r-1,r,r+1]:
        if i >= 0 and i < rows:
            for j in [c-1,c,c+1]:
                if j >= 0 and j < cols:
                    count += int(board[i,j])
    return count

In [4]:
def minesweepMatrix(dimensions):
    rows,cols = dimensions
    size = rows * cols
    A = np.zeros([size,size],dtype=int)
    for rA in range(size):
        for cA in range(size):
            inRow, inCol = divmod(rA,cols)
            outRow, outCol = divmod(cA,cols)
            A[rA,cA] = abs(inRow-outRow) <= 1 and abs(inCol-outCol) <= 1
    return(A)

In [5]:
# Converts a board of mines into a board of mine counts
'''
def boardMineCounts_(board):
    mineInfo = np.zeros(board.shape, dtype = int)
    rows, cols = board.shape
    for i in range(rows):
        for j in range(cols):
            mineInfo[i,j] = countMines(board,i,j)
    return mineInfo
'''
def boardMineCounts(board):
    return(minesweepMatrix(board.shape).dot(board.flatten()).reshape(board.shape))

In [6]:
def boardPartialMineCounts(board):
    result = boardMineCounts(board)
    for index, x in np.ndenumerate(board):
        if x: result[index] = -1
        elif r.uniform(0, 1) < missingProbability: result[index] = -1
    return result

In [7]:
# Generates a random training batch of size n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        batch_xs.append(counts.flatten())
        batch_ys.append(board.flatten().astype(float))
    return (np.asarray(batch_xs), np.asarray(batch_ys))

In [8]:
tightness = 5

In [9]:
# Create the model
rows, cols = dimensions
size = rows*cols
x = tf.placeholder(tf.int32, [None, size], name="x")
xoh = tf.reshape(tf.one_hot(x,9), [-1, size*9])
W = tf.Variable(tf.random_normal([size*9, size], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size], stddev=0.01), name="b")
y = tf.sigmoid(tightness*(tf.matmul(xoh, W) + b))

In [10]:
#predictedMines = tf.to_int32(y > 0.5)
M = tf.constant(minesweepMatrix(dimensions), dtype=tf.float32, name = "M")
predictedCounts = tf.matmul(y, M)
mask = tf.to_float(x >= 0)
maskedPredictedCounts = (predictedCounts + 1) * mask - 1

In [11]:
# Loss function
mean_squared_error = tf.losses.mean_squared_error(labels=x, predictions=maskedPredictedCounts)

In [12]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*9, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(-b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('accuracy', mean_squared_error)

In [13]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(mean_squared_error)

In [14]:
# Create session and initialise or restore stuff
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)
tf.global_variables_initializer().run()

In [15]:
# Restore model?
#saver.restore(sess, "./saves2/model-4000")

In [ ]:
# Train
for iteration in range(100001):
    batch_xs, _ = next_training_batch(100)
    summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs})
    writer.add_summary(summary, iteration)
    if iteration % 100 == 0:
        acc = sess.run(mean_squared_error, feed_dict={x: batch_xs})
        print('Accuracy at step %s: %s' % (iteration, acc))
    if iteration % 1000 == 0:
        save_path = saver.save(sess, './saves2/model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

Accuracy at step 0: 2.75703
Model saved in file: ./saves2/model-0
Accuracy at step 100: 0.333399
Accuracy at step 200: 0.218264
Accuracy at step 300: 0.168588
Accuracy at step 400: 0.136333
Accuracy at step 500: 0.120786
Accuracy at step 600: 0.105399
Accuracy at step 700: 0.0953765
Accuracy at step 800: 0.0877034
Accuracy at step 900: 0.0784798
Accuracy at step 1000: 0.0772322
Model saved in file: ./saves2/model-1000
Accuracy at step 1100: 0.0698182
Accuracy at step 1200: 0.0701031


In [ ]:
# Test trained model
batch_xs, _ = next_training_batch(1000)
print(sess.run(mean_squared_error, feed_dict={x: batch_xs}))

In [ ]:
# Run a single randomised test
mineCounts, mines = next_training_batch(1)

print("mines")
print(mines.astype(int).reshape(dimensions))

print("predicted mines")
result = sess.run(y, feed_dict={x: mineCounts})
predictions = (result > 0.5).astype(int)
print(predictions.reshape(dimensions))

print("errors")
print((predictions != mines.astype(int)).astype(int).sum())

print("----")

print("mine counts")
print(mineCounts.astype(int).reshape(dimensions))

print("predicted mine counts")
print(boardMineCounts(predictions.reshape(dimensions)))

print("errors")
print((mineCounts.astype(int).reshape(dimensions) != boardMineCounts(predictions.reshape(dimensions))).astype(int).sum())

In [ ]:
print(sess.run(-b))

In [ ]:
indices = [0, 1, 2]
depth = 3
tf.one_hot(indices, depth)